# Load functions

In [1]:
%run "C:/Users/asclo/Desktop/HHS/NIH Dashboard/Python Notebooks/Final Deliverables/FunctionsForAnalysis.py"

In [2]:
def testing_matching_source(known_annotations, annotations_to_test, graph_network):

    """  This allows you to test a dataset of annotations, with multiple text_ids, against

        This ONLY tests exact matches in the text_id and the references in the known annotations.

        Outputs are as follows:
        - output[0] - exact matches between the test annotators and known annotators
        - output[1] - exact and relational matches between the test annotators and known annotators
        - output[2] - Any annotations we tried to test, but did not have known annotators for

    """

    unique_text_sources = annotations_to_test['text_id'].drop_duplicates()

    #one dataset for direct matches and one of matches with relatives
    exact_final_data = pd.DataFrame()
    related_final_data = pd.DataFrame()
    
    list_for_issues = []

    for i in unique_text_sources:
        #one_source = unique_text_sources.iloc[i]
        one_test_set = annotations_to_test[annotations_to_test['text_id'] == i]

        one_source_for_known_annotators =  'PMID:' + i 

        one_known_set = known_annotations[known_annotations['reference'] == one_source_for_known_annotators]


        if len(one_known_set) == 0:
            
            list_for_issues.append(i)

        else:

            exact = one_annotation_direct_matching(one_known_set, one_test_set,  graph_network)
            exact['matching_to'] = i
            exact_final_data = exact_final_data.append(exact)

            related = one_annotation_matching_relatives(one_known_set, one_test_set, graph_network)
            related['matching_to'] = i
            related_final_data = related_final_data.append(related)
            
    
    return [exact_final_data, related_final_data, list_for_issues]


### Explaination of the testing_matching_source function

This function has a loop that takes each pubmed_id, extracts the hpo code assocaited directly with that pubmed_id, and compares themto:
- one_annotation_direct_matching function for exact matches
- one_annotation_matching_relatives function for exact and related matches

Both of these datasets can be inputed into the scoring function.


# Run Analysis

In [3]:
# graph database
g = hpo_hierarchy_graph_load()
phenotypic_abnormality_hpos = graph_phenotypic_abnormality(g)

In [4]:
# HPO annotations file- Our current Gold Standard
hpo_known_annotations = get_hpo_annotations_and_clean()
hpo_known_annotations = hpo_known_annotations[hpo_known_annotations['hpo'].isin(phenotypic_abnormality_hpos)]
hpo_known_annotations.head(2)

,#disease-db,disease-identifier,disease-name,negation,hpo,reference,evidence-code,onset,frequencyHPO,modifier,sub-ontology,alt-names,curators,frequencyRaw,sex,uniqueid
0,DECIPHER,1,Wolf-Hirschhorn Syndrome,NaN,HP:0000252,DECIPHER:1,IEA,NaN,NaN,,P,WOLF-HIRSCHHORN SYNDROME,HPO:skoehler[2013-05-29],-,-,HP:0000252DECIPHER:1
1,DECIPHER,1,Wolf-Hirschhorn Syndrome,NaN,HP:0001249,DECIPHER:1,IEA,NaN,NaN,,P,WOLF-HIRSCHHORN SYNDROME,HPO:skoehler[2013-05-29],-,-,HP:0001249DECIPHER:1


In [16]:
#Abstracts to Annotate
abstracts = pd.read_csv('C:/Users/asclo/Desktop/HHS/NIH Dashboard/Python Notebooks/Final Deliverables/All pmid abstracts from hpo annotations.csv',sep=',', index_col = False)
abstracts = abstracts.rename(columns = {'Pubmed_ID': 'text_id', 'Abstract': 'text'})

# pubmed ids that we know are case reports
# Qian provided this list
case_studies_pmid = pd.read_csv('C:/Users/asclo/Desktop/HHS/NIH Dashboard/Python Notebooks/Final Deliverables/Step1_extra - Full Qian Case Study List.csv', sep=',', index_col = False)

abstracts_for_analysis = abstracts[abstracts['text_id'].astype(int).isin(case_studies_pmid['PMID'])]

#making sure abstracts are in the known annotaions
abstracts_for_analysis['pmid'] = 'PMID:' + abstracts_for_analysis['text_id'].astype(str)
abstracts_for_analysis = abstracts_for_analysis[abstracts_for_analysis['pmid'].isin(hpo_known_annotations['reference'])]

abstracts_for_analysis = abstracts.iloc[0:100, :]

abstracts_for_analysis.head(3)

,text_id,Title,text,PubType
0,17088400,"Recurrent infections, hypotonia, and mental re...",OBJECTIVE: Our goal was to describe the neurol...,"research support, non-u.s. gov't; journal arti..."
1,19133692,15q overgrowth syndrome: a newly recognized ph...,Trisomy and tetrasomy of distal chromosome 15q...,journal article
2,10951463,Tetrasomy 15q25-->qter: cytogenetic and molecu...,Tetrasomy for the distal long arm of chromosom...,journal article; case reports


# Mondo Example

In [32]:
mondo_example = get_all_mondo_annotations(abstracts_for_analysis)
mondo_example = cleaning_mondo_to_hpo(mondo_example)

#We want to make sure the sample hpos are within phenotypic_abnormality
mondo_example = mondo_example[mondo_example['hpo'].isin(phenotypic_abnormality_hpos)]

In [33]:
one = testing_matching_source(hpo_known_annotations, mondo_example, g)

In [34]:
scoring(one[0]) # annotations that search for exact matches between hpo codes

,ScoringType,Annotations_to_Test,Known_Annotations,Accurately_Predicted,Accuracy,Precision,Recall,F1_Score,True_Positive,False_Positive,False_Negative,True_Negative
0,Direct,304,460,89,0.131852,0.292763,0.193478,0.232984,89,215,371,0


In [35]:
scoring(one[1]) # annotations that search for exact as well as child and parent matches

,ScoringType,Annotations_to_Test,Known_Annotations,Accurately_Predicted,Accuracy,Precision,Recall,F1_Score,True_Positive,False_Positive,False_Negative,True_Negative
0,Relative,305,460,99,0.148649,0.32459,0.215217,0.258824,99,206,361,0


In [36]:
one[2] # any text ids that did not match a known annotator reference

['9171831', '9781021']

# MetaMap Example

In [37]:
#MetaMap Example
metamap_example = get_all_metamap_annotations(abstracts_for_analysis)
metamap_example = cleaning_metamap_adding_hpo(metamap_example, g)

#We want to make sure the sample hpos are within phenotypic_abnormality
metamap_example = metamap_example[metamap_example['hpo'].isin(phenotypic_abnormality_hpos)]

In [38]:
one = testing_matching_source(hpo_known_annotations, metamap_example, g)

In [39]:
scoring(one[0]) # annotations that search for exact matches between hpo codes

,ScoringType,Annotations_to_Test,Known_Annotations,Accurately_Predicted,Accuracy,Precision,Recall,F1_Score,True_Positive,False_Positive,False_Negative,True_Negative
0,Direct,365,479,84,0.110526,0.230137,0.175365,0.199052,84,281,395,0


In [40]:
scoring(one[1]) # annotations that search for child and parent matches

,ScoringType,Annotations_to_Test,Known_Annotations,Accurately_Predicted,Accuracy,Precision,Recall,F1_Score,True_Positive,False_Positive,False_Negative,True_Negative
0,Relative,368,480,108,0.145946,0.293478,0.225,0.254717,108,260,372,0


In [41]:
one[2]

['21325395', '9171831', '9781021']

# Testing the MetaMap Categories

In [42]:
#MetaMap Example
metamap_example = get_all_metamap_annotations(abstracts_for_analysis)
metamap_example = cleaning_metamap_adding_hpo(metamap_example, g)

#We want to make sure the sample hpos are within phenotypic_abnormality
metamap_example = metamap_example[metamap_example['hpo'].isin(phenotypic_abnormality_hpos)]


# dsyn Disease or Syndrome
dsyn = metamap_example[metamap_example['semanticTypes'].apply(lambda x: 'dsyn' in x)]
# fndg Finding
fndg = metamap_example[metamap_example['semanticTypes'].apply(lambda x: 'fndg' in x)]
# sosy Sign or Symptom
sosy = metamap_example[metamap_example['semanticTypes'].apply(lambda x: 'sosy' in x)]

### Testing dsyn

In [43]:
dsyn_testing = testing_matching_source(hpo_known_annotations, dsyn, g)
scoring(dsyn_testing[0]) # annotations that search for exact matches between hpo codes

,ScoringType,Annotations_to_Test,Known_Annotations,Accurately_Predicted,Accuracy,Precision,Recall,F1_Score,True_Positive,False_Positive,False_Negative,True_Negative
0,Direct,111,221,21,0.067524,0.189189,0.095023,0.126506,21,90,200,0


In [44]:
scoring(dsyn_testing[1])  # annotations that search for child and parent matches

,ScoringType,Annotations_to_Test,Known_Annotations,Accurately_Predicted,Accuracy,Precision,Recall,F1_Score,True_Positive,False_Positive,False_Negative,True_Negative
0,Relative,111,221,32,0.106667,0.288288,0.144796,0.192771,32,79,189,0


### Testing fndg

In [45]:
fndg_testing = testing_matching_source(hpo_known_annotations, fndg, g)
scoring(fndg_testing[0]) # annotations that search for exact matches between hpo codes

,ScoringType,Annotations_to_Test,Known_Annotations,Accurately_Predicted,Accuracy,Precision,Recall,F1_Score,True_Positive,False_Positive,False_Negative,True_Negative
0,Direct,69,304,22,0.062678,0.318841,0.072368,0.117962,22,47,282,0


In [46]:
scoring(fndg_testing[1])  # annotations that search for child and parent matches

,ScoringType,Annotations_to_Test,Known_Annotations,Accurately_Predicted,Accuracy,Precision,Recall,F1_Score,True_Positive,False_Positive,False_Negative,True_Negative
0,Relative,70,304,25,0.071633,0.357143,0.082237,0.13369,25,45,279,0


### Testing sosy

In [47]:
sosy_testing = testing_matching_source(hpo_known_annotations, sosy, g)
scoring(sosy_testing[0]) # annotations that search for exact matches between hpo codes

,ScoringType,Annotations_to_Test,Known_Annotations,Accurately_Predicted,Accuracy,Precision,Recall,F1_Score,True_Positive,False_Positive,False_Negative,True_Negative
0,Direct,18,83,5,0.052083,0.277778,0.060241,0.09901,5,13,78,0


In [48]:
scoring(sosy_testing[1])  # annotations that search for child and parent matches

,ScoringType,Annotations_to_Test,Known_Annotations,Accurately_Predicted,Accuracy,Precision,Recall,F1_Score,True_Positive,False_Positive,False_Negative,True_Negative
0,Relative,18,83,6,0.063158,0.333333,0.072289,0.118812,6,12,77,0
